In [ ]:
'''
Script para carregar e tratar os dados da base da Olist 
Link: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce
'''

In [1]:
# Install
!pip install kagglehub

In [11]:
!pip install pymysql

In [1]:
# Importação de bibliotecas
import kagglehub
import pandas as pd
import pandas as pd
from sqlalchemy import create_engine
from kaggle.api.kaggle_api_extended import KaggleApi

In [5]:
# Chamando API e autenticando
api = KaggleApi()
api.authenticate()

In [13]:
# Download da base de dados 
path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")
print("Path to dataset files:", path)

100%|█████████████████████████████████████████████████████████████████████████████| 42.6M/42.6M [00:03<00:00, 11.5MB/s]

Extracting files...


Path to dataset files: C:\Users\renan\.cache\kagglehub\datasets\olistbr\brazilian-ecommerce\versions\2


In [3]:
# Leitura de todas as tabelas da base de dados
customer = pd.read_csv('olist_customers_dataset.csv')
geolocation = pd.read_csv('olist_geolocation_dataset.csv')
items = pd.read_csv('olist_order_items_dataset.csv')
payments = pd.read_csv('olist_order_payments_dataset.csv')
reviews = pd.read_csv('olist_order_reviews_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')
product_category = pd.read_csv('product_category_name_translation.csv')

In [4]:
# TRATAMENTO DO DATAFRAME ORDERS

# Transformando todas a colunas de data para o formato datetime
orders['order_purchase_timestamp']      = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_approved_at']             = pd.to_datetime(orders['order_approved_at'])
orders['order_delivered_carrier_date']  = pd.to_datetime(orders['order_delivered_carrier_date'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date'])

# Confirmando se todos os horários da coluna order_estimated_delivery_date são iguais
orders['order_estimated_delivery_date'].dt.time.unique()

# Alterando os nomes das colunas 
orders.rename(columns={'order_purchase_timestamp': 'order_purchase_date_time'}, inplace=True)
orders.rename(columns={'order_approved_at': 'order_approved_at_date_time'}, inplace=True)
orders.rename(columns={'order_delivered_carrier_date': 'order_delivered_carrier_date_time'}, inplace=True)
orders.rename(columns={'order_delivered_customer_date': 'order_delivered_customer_date_time'}, inplace=True)

# Criando colunas de Data
orders['order_purchase_date']           = orders['order_purchase_date_time'].dt.date
orders['order_approved_at_date']        = orders['order_approved_at_date_time'].dt.date
orders['order_delivered_carrier_date']  = orders['order_delivered_carrier_date_time'].dt.date
orders['order_delivered_customer_date'] = orders['order_delivered_customer_date_time'].dt.date

# Calculo do tempo de entrega
orders['delivery_days'] =  orders['order_delivered_customer_date'] - orders['order_purchase_date']

# Flag de pedidos atrasados
orders['late_order'] =  orders['order_delivered_customer_date'] > orders['order_estimated_delivery_date']

In [5]:
# TRATAMENTO DO DATAFRAME PRODUCTS

#Check de nulos
#display(products[products['product_category_name'].isnull()])

#Preenchendo nulos
products['product_category_name'] = products['product_category_name'].fillna('Outros')

#Alteração do tipo objeto para string
for col in products.select_dtypes(include = 'object').columns:
    products[col] = products[col].apply(lambda x: str(x) if pd.notnull(x) else None)

#Outras formas de fazer:  
    #products['product_id'] = products['product_id'].astype(str)
    #products['product_category_name'] = products['product_category_name'].astype(str)

In [6]:
# TRATAMENTO DO DATAFRAME CUSTOMER

#Verificando duplicidades de customer_unique_id
customer_id_duplicados = customer.groupby('customer_unique_id')['customer_id'].count().reset_index(name='quantidade')
customer_id_duplicados[customer_id_duplicados['quantidade'] > 1]

#Exclusão dos casos duplicados com menor frquência de ocorrência 
df_customer = (
    customer.groupby(['customer_unique_id', 'customer_id'])
    .size()
    .reset_index(name='frequencia')
    .sort_values(['customer_unique_id', 'frequencia'], ascending=[True, False])
    .drop_duplicates(subset='customer_unique_id')
)

customer = customer[customer['customer_id'].isin(df_customer['customer_id']) ]

In [8]:
#TRATAMENTO DO DATAFRAME PRODUCT_CATEGORY_NAME

#Check de nulos
display(product_category[product_category['product_category_name'].isnull()])

#Alteração do tipo objeto para string
for col in product_category.select_dtypes(include = 'object').columns:
    product_category[col] = product_category[col].apply(lambda x: str(x) if pd.notnull(x) else None)

#Conferindo a mudança feita acima: 
print(product_category['product_category_name'].apply(type).value_counts())

,product_category_name,product_category_name_english


product_category_name
<class 'str'>    71
Name: count, dtype: int64


In [9]:
#TRATAMENTO DO DATAFRAME SELLERS

#Alteração do tipo objeto para string
for col in sellers.select_dtypes(include = 'object').columns:
    sellers[col] = sellers[col].apply(lambda x: str(x) if pd.notnull(x) else None)

In [20]:
# Configurar conexão com MySQL
usuario = 'root'
senha = '....'
host = 'localhost'
porta = '3306'
banco = 'olist'

# Criando a engine de conexão
engine = create_engine(f'mysql+pymysql://{usuario}:{senha}@{host}:{porta}/{banco}')

In [24]:
#Inclui tabelas no banco criado acima
customer.to_sql('customer', engine, if_exists='replace', index=False)
geolocation.to_sql('geolocation', engine, if_exists='replace', index=False)
items.to_sql('items', engine, if_exists='replace', index=False)
payments.to_sql('payments', engine, if_exists='replace', index=False)
reviews.to_sql('reviews', engine, if_exists='replace', index=False)
orders.to_sql('orders', engine, if_exists='replace', index=False)
products.to_sql('products', engine, if_exists='replace', index=False)
sellers.to_sql('sellers', engine, if_exists='replace', index=False)
product_category.to_sql('product_category', engine, if_exists='replace', index=False)


3095